In [1]:
using TSSM
include("time_stepper.jl");
using PyPlot
#using CPUTime

 *** OPENMP n_threads =           8


In [2]:
cubic_coupling = -1.0
xmin=-16.0
xmax=16.0
Nx=1024

m=Schroedinger1D(Nx,xmin, xmax,cubic_coupling=cubic_coupling)
psia=wave_function(m)
psi_ref=wave_function(m)
psi=wave_function(m)

TSSM.WfSchroedinger1D{Float64}(Ptr{Void} @0x000000000391a3f0,TSSM.Schroedinger1D{Float64}(Ptr{Void} @0x00000000027e9ee0))

In [3]:
const a=[2.0, 2.0,  0.0,  0.0] #aplitude
const b=[1.0,-3.0,  1.0, -9.0] #speed
const c=[5.0,-5.0,  0.0, -10.0] #offset x

function soliton(x)
    (a[1]/cosh(a[1]*(x-c[1]))) * exp(-b[1]*x*1im)+
    (a[2]/cosh(a[2]*(x-c[2]))) * exp(-b[2]*x*1im)+
    (a[3]/cosh(a[3]*(x-c[3]))) * exp(-b[3]*x*1im)+
    (a[4]/cosh(a[4]*(x-c[4]))) * exp(-b[4]*x*1im)
end

soliton (generic function with 1 method)

In [4]:
#some schemes
#Emb 4/3 AK p
embedded_scheme_43 = EmbeddedScheme(
          ( 0.125962888700250514,  0.333588446797901933,
            0.751193431379145450, -0.338296598434303506,
            0.127551831557005609,  0.127551831557005609,
           -0.338296598434303506,  0.751193431379145450,
            0.333588446797901933,  0.125962888700250514 ),

          ( 0.125962888700250514,  0.333588446797901933,
            0.751193431379145450, -0.338296598434303506,
            0.0                 ,  0.261153550449697153,
           -0.242703571757396124,  0.596114052266110425,
            0.365547251678000160,  0.147440548920593995 ),

            4 )
#PP 3/4 A
palindromic_scheme_34 = PalindromicScheme( 
          ( 0.268330095781759925,  0.919661523017399857, 
           -0.187991618799159782, -0.187991618799159782, 
            0.919661523017399857,  0.268330095781759925 ),
            3 )
#PP 5/6 A
palindromic_scheme_56 = PalindromicScheme(
          ( 0.201651044312324230,   0.578800656272664932, 
            0.562615975356569200,   0.273128836056524479, 
            0.253874038247554845,  -0.102733803148432142, 
           -0.835351693190370636,   0.068014946093165092, 
            0.068014946093165092,  -0.835351693190370636,
           -0.102733803148432142,   0.253874038247554845, 
            0.273128836056524479,   0.562615975356569200, 
            0.578800656272664932,   0.201651044312324230 ),            
            5 )
#defectbased
embedded_scheme_43_D = DefectBasedScheme(
            embedded_scheme_43.scheme2,
            embedded_scheme_43.order-1 )
palindromic_scheme_34_D = DefectBasedScheme( 
            palindromic_scheme_34.scheme,
            palindromic_scheme_34.order )
palindromic_scheme_56_D = DefectBasedScheme(
            palindromic_scheme_56.scheme,            
            palindromic_scheme_56.order )
schemes=[embedded_scheme_43  ,palindromic_scheme_34  ,palindromic_scheme_56,
         embedded_scheme_43_D,palindromic_scheme_34_D,palindromic_scheme_56_D];

In [10]:
function equidistant_time_stepper(psi::WaveFunction, t0::Real, tend::Real, 
                         dt::Real, es::EmbeddedScheme, 
                         operator_sequence="AB")
    equidistant_time_stepper(psi, t0, tend, dt, es.scheme2,
                         operator_sequence)
end 
function equidistant_time_stepper(psi::WaveFunction, t0::Real, tend::Real, 
                         dt::Real, es::DefectBasedScheme, 
                         operator_sequence="AB")
    equidistant_time_stepper(psi, t0, tend, dt, es.scheme,
                         operator_sequence)
end 
function equidistant_time_stepper(psi::WaveFunction, t0::Real, tend::Real, 
                         dt::Real, ps::PalindromicScheme, 
                         operator_sequence="AB")
    equidistant_time_stepper(psi, t0, tend, dt, ps.scheme,
                         operator_sequence)
end 
function step!(psi::WaveFunction, dt::Real, scheme::EmbeddedScheme, operator_sequence="AB")
    step!(psi,dt,scheme.scheme2,operator_sequence)
end
function step!(psi::WaveFunction, dt::Real, scheme::PalindromicScheme, operator_sequence="AB")
    step!(psi,dt,scheme.scheme,operator_sequence)
end
function step!(psi::WaveFunction, dt::Real, scheme::DefectBasedScheme, operator_sequence="AB")
    step!(psi,dt,scheme.scheme,operator_sequence)
end

step! (generic function with 9 methods)

In [6]:
function my_savefig(scheme,schemes,mytime,steps_a,nsteps_a)
    figure(1)
    hold(false)
            plot(mytime[1:nsteps_a-1], steps_a[1:nsteps_a-1])
            xlabel("t")
            ylabel("stepsize")
            if scheme==schemes[1]
                savefig("Emb43AKp_$(tol)_step.png", bbox_inches="tight")
            end
            if scheme==schemes[2]
                savefig("PP34A$(tol)_step.png", bbox_inches="tight")
            end
            if scheme==schemes[3]
                savefig("PP56A_$(tol)_step.png", bbox_inches="tight")
            end
            if scheme==schemes[4]
                savefig("Emb43AKp_defect_$(tol)_step.png", bbox_inches="tight")
            end
            if scheme==schemes[5]
                savefig("PP34A_defect$(tol)_step.png", bbox_inches="tight")
            end
            if scheme==schemes[6]
                savefig("PP56A_defect_$(tol)_step.png", bbox_inches="tight")
            end
end
function calc_stepmin(psi_ref,psia,psi,scheme,soliton)
    h=1.0
    chg=h/2
    #@printf("||psi_a-psi_ref||=%5.3E ,h=%5.3E \n",distance(psia,psi_ref),h)
    set!(psi, soliton) 
    while chg>1e-3 || distance(psia,psi_ref) < distance(psi,psi_ref)
        set!(psi, soliton) 
        time = @elapsed for t in equidistant_time_stepper(psi, t0, tend, h, scheme, "AB")
        end
        #@printf("||psi-psi_ref||=%5.3E,  h=%5.3E \n",distance(psi,psi_ref),h)
        if distance(psia,psi_ref) < distance(psi,psi_ref)
            h=h-chg
        else
            h=h+chg
        end
        chg=chg/2
    end
    #@printf("||psi-psi_ref||=%5.3E,  h=%20.18E \n",distance(psi,psi_ref),h)
    return h
end
function my_latex(scheme,schemes,tol,nsteps_a,nsteps,time_a,time)
            if scheme==schemes[1]
            @printf("\\texttt{Emb 4/3 AK p}, TOL = \$ 10^{%d} \$ & \$ %d \$ & \$ %d \$ & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \n",tol,nsteps_a,nsteps,time_a,time)
        end
        if scheme==schemes[2]
            @printf(    "\\texttt{PP 3/4 A}, TOL = \$ 10^{%d} \$ & \$ %d \$ & \$ %d \$ & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \n",tol,nsteps_a,nsteps,time_a,time)
        end
        if scheme==schemes[3]
            @printf(    "\\texttt{PP 5/6 A}, TOL = \$ 10^{%d} \$ & \$ %d \$ & \$ %d \$ & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \\hline \n",tol,nsteps_a,nsteps,time_a,time)
        end
        if scheme==schemes[4]
            @printf("\\texttt{Emb 4/3 AK p (defect)}, TOL = \$ 10^{%d} \$ & \$ %d \$ & \$ %d \$ & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \n",tol,nsteps_a,nsteps,time_a,time)
        end
        if scheme==schemes[5]
            @printf(    "\\texttt{PP 3/4 A (defect)}, TOL = \$ 10^{%d} \$ & \$ %d \$ & \$ %d \$ & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \n",tol,nsteps_a,nsteps,time_a,time)
        end
        if scheme==schemes[6]
            @printf(    "\\texttt{PP 5/6 A (defect)}, TOL = \$ 10^{%d} \$ & \$ %d \$ & \$ %d \$ & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \\hline \n",tol,nsteps_a,nsteps,time_a,time)
        end
end

my_latex (generic function with 1 method)

In [7]:
tend=5.0;
t0=0.0;
tols=[-5,-8,-10];
out=0;
stepmin_ex=0;
latex=1;

In [8]:
set!(psi_ref, soliton) #compute ref_solution
@time for t in adaptive_time_stepper(psi_ref, t0, tend, 0.01, 1e-12, palindromic_scheme_56, "AB")
end

  2.981041 seconds (292.03 k allocations: 8.054 MB, 0.15% gc time)


In [9]:
@printf("Method & \\# steps adaptive & \\# steps equidist & time adaptive & time equidist \\\\ \\hline \n")
maxsteps=10^6
nrep=25
for tol in tols
    for scheme in schemes
        steps_a=zeros(maxsteps)
        mytime=zeros(maxsteps)
        time_a=0.0
        nsteps_a=0
        for n in 1:nrep #average time over nrep runs
            told=0.0
            nsteps_a=0
            set!(psia, soliton)
            time_a =time_a+ @elapsed  for t in adaptive_time_stepper(psia, t0, tend, 0.01, 10.0^tol, scheme, "AB")
                    nsteps_a=nsteps_a+1
                    steps_a[nsteps_a]=t-told
                    mytime[nsteps_a]=t
                    told=t    
            end
        end
        time_a=time_a/nrep
        stepmin=minimum(steps_a[1:nsteps_a-1])
        if out==1
            my_savefig(scheme,schemes,mytime,steps_a,nsteps_a)
        end
        if stepmin_ex==1
            stepmin=calc_stepmin(psi_ref,psia,psi,scheme,soliton)
        end 
        steps=zeros(maxsteps)
        mytime=zeros(maxsteps)
        nsteps=0
        time=0.0
        for n in 1:nrep #average time over nrep runs
            told=0.0
            nsteps=0
            set!(psi, soliton)
            time = time + @elapsed  for t in equidistant_time_stepper(psi, t0, tend, stepmin, scheme, "AB")
                nsteps=nsteps+1
                steps[nsteps]=t-told
                mytime[nsteps]=t
                told=t 
            end
        end
        time=time/nrep
        if latex==1
            my_latex(scheme,schemes,tol,nsteps_a,nsteps,time_a,time)
        end
    end
end

Method & \# steps adaptive & \# steps equidist & time adaptive & time equidist \\ \hline 
\texttt{Emb 4/3 AK p}, TOL = $ 10^{-5} $ & $ 275 $ & $ 1070 $ & $ 0.140 $ & $ 0.322 $ \\ 
\texttt{PP 3/4 A}, TOL = $ 10^{-5} $ & $ 438 $ & $ 1749 $ & $ 0.186 $ & $ 0.333 $ \\ 
\texttt{PP 5/6 A}, TOL = $ 10^{-5} $ & $ 156 $ & $ 501 $ & $ 0.167 $ & $ 0.240 $ \\ \hline 
\texttt{Emb 4/3 AK p (defect)}, TOL = $ 10^{-5} $ & $ 265 $ & $ 1034 $ & $ 0.159 $ & $ 0.261 $ \\ 
\texttt{PP 3/4 A (defect)}, TOL = $ 10^{-5} $ & $ 438 $ & $ 1749 $ & $ 0.187 $ & $ 0.305 $ \\ 
\texttt{PP 5/6 A (defect)}, TOL = $ 10^{-5} $ & $ 189 $ & $ 733 $ & $ 0.203 $ & $ 0.350 $ \\ \hline 
\texttt{Emb 4/3 AK p}, TOL = $ 10^{-8} $ & $ 1536 $ & $ 6005 $ & $ 0.716 $ & $ 1.654 $ \\ 
\texttt{PP 3/4 A}, TOL = $ 10^{-8} $ & $ 2478 $ & $ 9900 $ & $ 0.924 $ & $ 1.606 $ \\ 
\texttt{PP 5/6 A}, TOL = $ 10^{-8} $ & $ 504 $ & $ 2005 $ & $ 0.494 $ & $ 0.901 $ \\ \hline 
\texttt{Emb 4/3 AK p (defect)}, TOL = $ 10^{-8} $ & $ 1495 $ & $ 5847 $ & $ 

In [8]:
    nstep=10000
    tol=1e-5
    dt=0.01
    @printf("Method & %d steps without errorestimator & %d steps with errorestimator\\\\ \\hline \n",nstep,nstep)
    for scheme in schemes
        set!(psi, soliton)
        time1 = @elapsed  for t in 1:nstep
            step!(psi, dt, scheme,"AB")
        end
        psi2=clone(psi)
        time2 = @elapsed  for t in 1:nstep
        if typeof(scheme)==PalindromicScheme
           copy!(psi, psi2)
           step_palindromic!(psi, psi2, dt, scheme.scheme, "AB")
           err = 0.5*distance(psi, psi2)/tol
        end
        if typeof(scheme)==EmbeddedScheme
           copy!(psi, psi2)
           step_embedded!(psi, psi2, dt, scheme.scheme1, scheme.scheme2, "AB")
           err = distance(psi, psi2)/tol
        end  
        if typeof(scheme)==DefectBasedScheme
            step_defect_based!(psi, psi2, dt, scheme.scheme, "AB")
            err = dt*norm(psi2)/(scheme.order+1)/tol
        end
    end
        if scheme==schemes[1]
            @printf("\\texttt{Emb 4/3 AK p} & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \n",time1,time2)
        end
        if scheme==schemes[2]
            @printf(    "\\texttt{PP 3/4 A} & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \n",time1,time2)
        end
        if scheme==schemes[3]
        @printf(    "\\texttt{PP 5/6 A} & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \\hline \n",time1,time2)
        end
        if scheme==schemes[4]
            @printf("\\texttt{Emb 4/3 AK p (defect)} & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \n",time1,time2)
        end
        if scheme==schemes[5]
            @printf(    "\\texttt{PP 3/4 A (defect)} & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \n",time1,time2)
        end
        if scheme==schemes[6]
        @printf(    "\\texttt{PP 5/6 A (defect)} & \$ %5.3f \$ & \$ %5.3f \$ \\\\ \\hline \n",time1,time2)
        end
    end

Method & 10000 steps without errorestimator & 10000 steps with errorestimator\\ \hline 
\texttt{Emb 4/3 AK p} & $ 2.733 $ & $ 3.687 $ \\ 
\texttt{PP 3/4 A} & $ 1.647 $ & $ 3.013 $ \\ 
\texttt{PP 5/6 A} & $ 4.395 $ & $ 7.794 $ \\ \hline 
\texttt{Emb 4/3 AK p (defect)} & $ 2.742 $ & $ 6.006 $ \\ 
\texttt{PP 3/4 A (defect)} & $ 1.654 $ & $ 3.665 $ \\ 
\texttt{PP 5/6 A (defect)} & $ 4.374 $ & $ 9.243 $ \\ \hline 


In [9]:
t0 = 0
tend = 1
cubic_coupling = -1.0
xmin=-16.0
xmax=16.0
Nx=1024

const a1 = 2
const b1 = 1
const c1 = 0
function soliton(x, t)
    h = (a1^2 - b1^2)/2*t - b1*x
    (a1./cosh(a1*(b1*t+x-c1))).*exp(1im*h) 
end 

m=Schroedinger1D(Nx,xmin, xmax,cubic_coupling=cubic_coupling)
for scheme in [schemes[2],schemes[5],] 
psi = wave_function(m)
psi_ref = wave_function(m)
set!(psi, soliton, t0)        # initial data at t=t0
set!(psi_ref, soliton, tend)
    local_orders(psi, soliton, t0, tend-t0, scheme, operator_sequence="AB", rows=12)
end
;

             dt         err      p         err      p
------------------------------------------------------
  1   1.000e+00   1.174e+00          1.159e+00
  2   5.000e-01   3.684e-01   1.67   2.704e-01   2.10
  3   2.500e-01   8.816e-02   2.06   4.521e-02   2.58
  4   1.250e-01   1.357e-02   2.70   5.133e-03   3.14
  5   6.250e-02   1.336e-03   3.34   3.562e-04   3.85
  6   3.125e-02   1.000e-04   3.74   1.549e-05   4.52
  7   1.563e-02   6.708e-06   3.90   5.165e-07   4.91
  8   7.813e-03   4.294e-07   3.97   1.612e-08   5.00
  9   3.906e-03   2.702e-08   3.99   5.022e-10   5.00
 10   1.953e-03   1.692e-09   4.00   1.568e-11   5.00
 11   9.766e-04   1.058e-10   4.00   4.897e-13   5.00
 12   4.883e-04   6.612e-12   4.00   1.556e-14   4.98
             dt         err      p         err      p
------------------------------------------------------
  1   1.000e+00   1.174e+00          1.116e+00
  2   5.000e-01   3.684e-01   1.67   3.308e-01   1.76
  3   2.500e-01   8.816e-02   2.06   6.8